# The code in this notebook was used to create a .ped file from a .bed file

## Creating .ped files used for running GWAS on plink

The tarball contains data from the VASST study.  For new datasets, assuming there are .bed, .bim and .fam files, rename all their prefixes to template and replace the files in the "template" folder, and everything will work.

The idea is to convert a .bed file into a .ped file, then modify it so it will contain the cytokine level as "trait" instead of survival.  Afterwards, the .ped file can be repacked into a binary .bed file and a GWAS can be performed by plink.

In [ ]:
!tar -zxvf template.tar.gz

In [ ]:
# Convert the binary .bed file into a .ped file

!plink --bfile template/template --recode --tab --out template/template --noweb

In [ ]:
# Create a .map file (which are just the first 4 columns of the .bim file)
# This file is required to create a .bed file

!cut -f 1-4 template/template.bim >> template/template.map

In [ ]:
# Because the .ped file is large, and only the first few columns are required to be modified, the file is cut in 
# two (one with the first 6 columns, one with all the SNPs) and only the first few columns are modified to
# prevent loading the entire ~1 million columns

!cut -f 7- template/template.ped >> template/cut_template.ped

# Creating .bed files and running GWAS

In [ ]:
import pandas as pd 

# Assuming that the excel file is formatted like the excel of the VASST study, modification are unecessary
cytokines = pd.read_excel("VASST_data/Sepsis_patient_cytokine_levels.xlsx")

In [ ]:
# For a binary interpretation of cytokine levels, the mean is calculated and the trait is interpreted as either
# low levels or high levels of cytokine (above or below the mean)

mean_cytokine = cytokines.mean(0)

In [ ]:
# Since the first 6 columns of the .ped file is the same as the .fam file, we can load either of the files

ped = pd.read_csv("template/template.fam", sep=' ', 
                  names=["fid", "iid", "father", "mother", "gender", "trait"])

In [ ]:
# This script is for modifying the .ped file and running an quantitative GWAS on plink.  The results obtained 
# were from a quantitative GWAS where the trait was the exact value of the cytokine level

# The commented segments are for a binary interpretation of cytokine levels

import math
import os
import subprocess

cwd = os.getcwd()

# This huge loop creates subdirectories in either 'continuous_cytokines' or 'binary_cytokines'
# corresponding to each cytokine.  Within the subdirectory will be the GWAS association results in .qassoc 
# or .assoc as well as a manhattan plot and a .bed file

for cytokine in mean_cytokine.iteritems():
    destination = os.path.join(cwd, 'continuous_cytokines/'+cytokine[0])
#     destination = os.path.join(cwd, 'binary_cytokines/'+cytokine[0])
    for index,row in ped.iterrows():
        cytokine_level = float(cytokines.loc[cytokines['SampleNumber'] == row['fid']][cytokine[0]].values)
        ### If there is no cytokine level, drop this row
        if math.isnan(cytokine_level):
            ped.drop(ped.index[index])
#         elif cytokine_level < mean_cytokine[cytokine[0]]:
#             ped.at[index, 'trait'] = 1
#         else:
#             ped.at[index, 'trait'] = 2
        else:
            ped.at[index, 'trait'] = cytokine_level
    if not os.path.exists(destination):
        os.makedirs(destination)
    ped.to_csv(os.path.join(destination, cytokine[0]+"_temp.ped"), 
               header=None, index=None, sep=' ', mode='a')
    subprocess.call(['cp', 'template/template.map', destination])
    subprocess.call(['mv', os.path.join(destination, 'template.map'), os.path.join(destination, cytokine[0]+".map")])
    paste = "paste -d' ' %s template/cut_template.ped >> %s" % (os.path.join(destination, cytokine[0]+"_temp.ped"), os.path.join(destination, cytokine[0]+".ped"))
    !{paste}
    bed = "plink --file %s --make-bed --out %s --noweb" % (os.path.join(destination, cytokine[0]), os.path.join(destination, cytokine[0]))
    gwas = "plink --bfile %s --assoc --out %s --noweb" % (os.path.join(destination, cytokine[0]), os.path.join(destination, cytokine[0]))
    !{bed}
    !{gwas}
    Rscript = "Rscript --vanilla ../../visualization/manhattan.R %s %s" % (os.path.join(destination, cytokine[0]+".qassoc"), os.path.join(destination, cytokine[0]+".png"))
#     Rscript = "Rscript --vanilla ../../visualization/manhattan.R %s %s" % (os.path.join(destination, cytokine[0]+".assoc"), os.path.join(destination, cytokine[0]+".png"))
    !{Rscript}
    delete = "rm %s" % os.path.join(destination, "*.ped")
    !{delete}